In [1]:
import datatable as dt
import pandas as pd
import numpy as np

# Problem: https://www.hackerearth.com/challenges/competitive/amexpert-code-lab/machine-learning/credit-card-default-risk-5-95cbc85f/

In [2]:
import warnings
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import KNNImputer
import pickle
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


# Credit: https://johaupt.github.io/scikit-learn/tutorial/python/data%20processing/ml%20pipeline/model%20interpretation/columnTransformer_feature_names.html
def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
    """

    # Remove the internal helper function
    #check_is_fitted(column_transformer)

    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names'):
            # >>> Change: Return input column names if no method avaiable
            # Turn error into a warning
            warnings.warn("Transformer %s (type %s) does not "
                          "provide get_feature_names. "
                          "Will return input column names if available"
                          % (str(name), type(trans).__name__))
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [f for f in column]

        return [f for f in trans.get_feature_names()]

    ### Start of processing
    feature_names = []

    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))

    for name, trans, column, _ in l_transformers:
        if type(trans) == Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names) == 0:
                _names = [f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))

    return feature_names


# Credit: https://stackoverflow.com/questions/60273501/column-specific-processing-in-an-sklearn-pipeline
def preprocess(url, train=True):
    df = dt.fread(url).to_pandas()
    df.set_index('customer_id', inplace=True)
    index_to_be_dropped = df[(df['gender'] == 'XNA') | (df['owns_car'] == '')].index
    df.drop(index_to_be_dropped, inplace=True)
    if train:
        target_col = df.columns[-1]
        y = df[target_col]
        y = pd.Series(map(int, y), index=y.index)
        X = df.drop(columns=['name', target_col])
    else:
        X = df.drop(columns=['name'])

    impute_transformer = Pipeline([('knn_impute', KNNImputer())])
    one_hot_transformer = Pipeline([('one_hot', OneHotEncoder(drop='first'))])
    scale_transformer = Pipeline([('standard_scale', StandardScaler())])

    null_cols = X.columns[X.isnull().any()]
    oh_cols, to_be_scaled_cols = [], []
    for col in X.columns:
        if X[col].nunique() > 3 and np.issubdtype(X[col].dtype, np.number):
            to_be_scaled_cols.append(col)
        else:
            oh_cols.append(col)
    if train:
        processor = ColumnTransformer([
            ('imputed', impute_transformer, null_cols),
            ('encoded', one_hot_transformer, oh_cols),
            ('scaled', scale_transformer, to_be_scaled_cols),
        ], remainder='passthrough')
        processor.fit(X)
        # Save to file in the current working directory
        with open('processor.pkl', 'wb') as file:
            pickle.dump(processor, file)
        X = pd.DataFrame(processor.transform(X), columns=get_feature_names(processor), index=y.index)
        return X, y
    else:
        # Load from file
        with open('processor.pkl', 'rb') as file:
            processor = pickle.load(file)
        X = pd.DataFrame(processor.transform(X), columns=get_feature_names(processor))
        return X

In [3]:
X_train, y_train = preprocess('train.csv', train=True)
X_test = preprocess('test.csv')

C:\Users\Admin\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Admin\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Admin\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Admin\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Admin\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\Admin\miniconda3\lib\site-packages\numpy\core\fromnumeric.py:86: R

In [4]:
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')
X_test = X_test[0].to_csv('X_test.csv')